<a href="https://colab.research.google.com/github/jjbmsda/EnsembleModel/blob/main/EnsembleModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# soundfile 설치
!pip -q install soundfile

In [3]:
# torchcodec 설치
!pip -q install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.9 MB/s eta 0:00:00


In [4]:
import torchaudio
from torchaudio.datasets import LIBRISPEECH

train_raw = LIBRISPEECH(
    root="/content/drive/MyDrive/datasets",
    url="dev-clean",
    download=False
)

test_raw = LIBRISPEECH(
    root="/content/drive/MyDrive/datasets",
    url="test-clean",
    download=False
)

In [ ]:
import os, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.models import resnet18, densenet121

import torchaudio
from torchaudio.datasets import LIBRISPEECH
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import top_k_accuracy_score


# =========================
# Colab RAM-safe settings
# =========================
# 하이퍼파라미터 세팅
BATCH_SIZE = 8
EPOCHS = 1
MAX_SPEAKERS = 50
TOPK = (1, 3)


def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)


def pad_trim_2d(spec: torch.Tensor, target_frames: int, pad_value: float = 0.0) -> torch.Tensor:
    T = spec.size(-1) #spec 의 마지막 차원 길이
    if T > target_frames: #dataset 에서 targetframe 을 받아와서 비교
        return spec[..., :target_frames]
    elif T < target_frames:
        return F.pad(spec, (0, target_frames - T), value=pad_value)
    return spec


class LibriSpeechSpeakerDataset(Dataset):
    def __init__(self, dataset, spk2idx, sample_rate=16000, n_mels=64, target_frames=256):
        self.dataset = dataset
        self.spk2idx = spk2idx
        self.sample_rate = sample_rate
        self.target_frames = target_frames
        self.melspec = MelSpectrogram(sample_rate=sample_rate, n_mels=n_mels)
        self.to_db = AmplitudeToDB(stype="power")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        waveform, sr, transcript, speaker_id, chapter_id, utterance_id = self.dataset[idx]
        spk = int(speaker_id)

        # max_speakers 밖 화자는 스킵
        if spk not in self.spk2idx:
            return None

        # mono
        if waveform.size(0) > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # resample to 16k
        if sr != self.sample_rate:
            waveform = torchaudio.functional.resample(waveform, sr, self.sample_rate)

        # mel -> db, then pad/trim time axis
        spec = self.to_db(self.melspec(waveform))     # [1, n_mels, T]
        spec = pad_trim_2d(spec, self.target_frames)  # [1, n_mels, target_frames]

        y = self.spk2idx[spk]
        return spec, torch.tensor(y, dtype=torch.long)


def collate_skip_none(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return None
    xs, ys = zip(*batch)
    return torch.stack(xs, dim=0), torch.stack(ys, dim=0)


class ResNetModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.net = resnet18(weights=None, num_classes=num_classes)
        self.net.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.net.maxpool = nn.Identity()

    def forward(self, x):
        return self.net(x)


class DenseNetModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.net = densenet121(weights=None)
        self.net.features.conv0 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.net.features.pool0 = nn.Identity()
        self.net.classifier = nn.Linear(self.net.classifier.in_features, num_classes)

    def forward(self, x):
        return self.net(x)


def train_one_epoch(model, loader, criterion, optimizer, device, use_amp, epoch, model_name="model"):
    model.train()
    total_loss, steps = 0.0, 0
    scaler = torch.amp.GradScaler("cuda") if use_amp else None

    for i, batch in enumerate(loader):
        if batch is None:
            continue
        x, y = batch
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad(set_to_none=True)

        if use_amp:
            with torch.amp.autocast("cuda"):
                logits = model(x)
                loss = criterion(logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

        total_loss += float(loss.item())
        steps += 1

        if (i + 1) % 50 == 0:
            print(
                f"[Epoch {epoch}][{model_name}] "
                f"step {i+1}/{len(loader)} "
                f"loss={total_loss/steps:.4f}"
            )

    return total_loss / max(1, steps)


@torch.no_grad()
def eval_model(model, loader, device, num_classes, topk=TOPK):
    model.eval()
    probs_all, targets_all = [], []

    for batch in loader:
        if batch is None:
            continue
        x, y = batch
        x = x.to(device)

        logits = model(x)
        probs = logits.softmax(dim=1).cpu().numpy()

        probs_all.append(probs)
        targets_all.append(y.numpy())

    if len(targets_all) == 0:
        return None

    probs_all = np.concatenate(probs_all, axis=0)
    targets_all = np.concatenate(targets_all, axis=0)
    preds = probs_all.argmax(axis=1)

    out = {
        "acc": accuracy_score(targets_all, preds),
        "macro_f1": f1_score(targets_all, preds, average="macro"),
        "top1": top_k_accuracy_score(targets_all, probs_all, k=1, labels=list(range(num_classes)))
    }
    for k in topk:
        if k <= num_classes:
            out[f"top{k}"] = top_k_accuracy_score(targets_all, probs_all, k=k, labels=list(range(num_classes)))
    return out


@torch.no_grad()
def eval_ensemble(rnet, dnet, loader, device, num_classes, topk=TOPK, alpha=0.8):
    rnet.eval()
    dnet.eval()
    probs_all, targets_all = [], []

    for batch in loader:
        if batch is None:
            continue
        x, y = batch
        x = x.to(device)

        # (중요) logits_r/logits_d를 먼저 계산해야 함
        logits_r = rnet(x)
        logits_d = dnet(x)

        # 가중 앙상블
        logits = alpha * logits_r + (1 - alpha) * logits_d
        probs = logits.softmax(dim=1).cpu().numpy()

        probs_all.append(probs)
        targets_all.append(y.numpy())

    if len(targets_all) == 0:
        return None

    probs_all = np.concatenate(probs_all, axis=0)
    targets_all = np.concatenate(targets_all, axis=0)
    preds = probs_all.argmax(axis=1)

    out = {
        "acc": accuracy_score(targets_all, preds),
        "macro_f1": f1_score(targets_all, preds, average="macro"),
        "top1": top_k_accuracy_score(targets_all, probs_all, k=1, labels=list(range(num_classes)))
    }
    for k in topk:
        if k <= num_classes:
            out[f"top{k}"] = top_k_accuracy_score(targets_all, probs_all, k=k, labels=list(range(num_classes)))
    return out


def split_indices(n, seed=42, train=0.8, val=0.1):
    idxs = list(range(n))
    rng = random.Random(seed)
    rng.shuffle(idxs)
    n_train = int(n * train)
    n_val = int(n * val)
    train_idxs = idxs[:n_train]
    val_idxs = idxs[n_train:n_train + n_val]
    test_idxs = idxs[n_train + n_val:]
    return train_idxs, val_idxs, test_idxs


def main():
    os.makedirs("./data", exist_ok=True)

    # dev-clean만 사용
    raw = LIBRISPEECH("./data", url="dev-clean", download=True)

    # speaker mapping (최대 MAX_SPEAKERS명만 사용)
    all_speakers = sorted({int(spk) for _, _, _, spk, *_ in raw})
    speakers = all_speakers[:MAX_SPEAKERS]
    spk2idx = {spk: i for i, spk in enumerate(speakers)}
    num_classes = len(speakers)

    print(f"Using speakers: {num_classes}/{len(all_speakers)} (MAX_SPEAKERS={MAX_SPEAKERS})")

    full_ds = LibriSpeechSpeakerDataset(raw, spk2idx, n_mels=64, target_frames=256)

    # split (발화 단위)
    tr, va, te = split_indices(len(full_ds), seed=42, train=0.8, val=0.1)
    train_ds = Subset(full_ds, tr)
    val_ds   = Subset(full_ds, va)
    test_ds  = Subset(full_ds, te)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    use_amp = (device.type == "cuda")

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_skip_none)
    val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_skip_none)
    test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_skip_none)

    rnet = ResNetModel(num_classes).to(device)
    dnet = DenseNetModel(num_classes).to(device)

    crit = nn.CrossEntropyLoss()
    opt_r = optim.Adam(rnet.parameters(), lr=1e-3)
    opt_d = optim.Adam(dnet.parameters(), lr=1e-3)

    for ep in range(1, EPOCHS + 1):
        loss_r = train_one_epoch(
            rnet, train_loader, crit, opt_r, device, use_amp,
            epoch=ep, model_name="ResNet"
        )

        loss_d = train_one_epoch(
            dnet, train_loader, crit, opt_d, device, use_amp,
            epoch=ep, model_name="DenseNet"
        )

        val_r = eval_model(rnet, val_loader, device, num_classes, topk=TOPK)
        val_d = eval_model(dnet, val_loader, device, num_classes, topk=TOPK)
        val_e = eval_ensemble(rnet, dnet, val_loader, device, num_classes, topk=TOPK, alpha=0.8)

        print(f"\n[Epoch {ep}/{EPOCHS}]")
        print(f"  ResNet   loss={loss_r:.4f}  val={val_r}")
        print(f"  DenseNet loss={loss_d:.4f}  val={val_d}")
        print(f"  Ensemble           val={val_e}")

    test_r = eval_model(rnet, test_loader, device, num_classes, topk=TOPK)
    test_d = eval_model(dnet, test_loader, device, num_classes, topk=TOPK)
    test_e = eval_ensemble(rnet, dnet, test_loader, device, num_classes, topk=TOPK, alpha=0.8)

    if test_r is None or test_d is None or test_e is None:
        print("\nTEST set이 비어있거나 모두 None으로 필터링됐어. (split/데이터 로딩 확인 필요)")
        return

    print("\n=== TEST RESULTS (dev-clean split) ===")
    print(f"ResNet   {test_r}")
    print(f"DenseNet {test_d}")
    print(f"Ensemble {test_e}")


if __name__ == "__main__":
    main()

Using speakers: 40/40 (MAX_SPEAKERS=50)
[Epoch 1][ResNet] step 50/271 loss=3.3546
[Epoch 1][ResNet] step 100/271 loss=2.9880
[Epoch 1][ResNet] step 150/271 loss=2.7235
[Epoch 1][ResNet] step 200/271 loss=2.4949
[Epoch 1][ResNet] step 250/271 loss=2.2784
[Epoch 1][DenseNet] step 50/271 loss=3.6242
[Epoch 1][DenseNet] step 100/271 loss=3.3382
[Epoch 1][DenseNet] step 150/271 loss=3.0942
[Epoch 1][DenseNet] step 200/271 loss=2.9085
[Epoch 1][DenseNet] step 250/271 loss=2.7663

[Epoch 1/1]
  ResNet   loss=2.2080  val={'acc': 0.5185185185185185, 'macro_f1': 0.471012906867878, 'top1': np.float64(0.5185185185185185), 'top3': np.float64(0.8037037037037037)}
  DenseNet loss=2.7120  val={'acc': 0.43703703703703706, 'macro_f1': 0.35431933093603574, 'top1': np.float64(0.43703703703703706), 'top3': np.float64(0.7)}
  Ensemble           val={'acc': 0.5555555555555556, 'macro_f1': 0.4918604140691743, 'top1': np.float64(0.5555555555555556), 'top3': np.float64(0.8333333333333334)}

=== TEST RESULTS (de

In [ ]:

import os, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.models import resnet18, densenet121

import torchaudio
from torchaudio.datasets import LIBRISPEECH
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB

from sklearn.metrics import accuracy_score, f1_score, roc_curve

# =========================
# ✅ Colab RAM-safe settings
# =========================
BATCH_SIZE = 8
EPOCHS = 1
MAX_SPEAKERS = 50

# Embedding config
EMB_DIM = 256
MARGIN = 0.2
SCALE = 30.0

# ✅ 앙상블 임베딩 비중 (ResNet 비중)
ALPHA = 0.8

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)

def pad_trim_2d(spec: torch.Tensor, target_frames: int, pad_value: float = 0.0) -> torch.Tensor:
    T = spec.size(-1)
    if T > target_frames:
        return spec[..., :target_frames]
    elif T < target_frames:
        return F.pad(spec, (0, target_frames - T), value=pad_value)
    return spec

class LibriSpeechSpeakerDataset(Dataset):
    def __init__(self, dataset, spk2idx, sample_rate=16000, n_mels=64, target_frames=256):
        self.dataset = dataset
        self.spk2idx = spk2idx
        self.sample_rate = sample_rate
        self.target_frames = target_frames
        self.melspec = MelSpectrogram(sample_rate=sample_rate, n_mels=n_mels)
        self.to_db = AmplitudeToDB(stype="power")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        waveform, sr, transcript, speaker_id, chapter_id, utterance_id = self.dataset[idx]
        spk = int(speaker_id)

        # MAX_SPEAKERS 밖 화자 스킵(매핑 없는 화자)
        if spk not in self.spk2idx:
            return None

        # mono
        if waveform.size(0) > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # resample
        if sr != self.sample_rate:
            waveform = torchaudio.functional.resample(waveform, sr, self.sample_rate)

        spec = self.to_db(self.melspec(waveform))     # [1, n_mels, T]
        spec = pad_trim_2d(spec, self.target_frames)  # [1, n_mels, target_frames]

        y = self.spk2idx[spk]
        return spec, torch.tensor(y, dtype=torch.long)

def collate_skip_none(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return None
    xs, ys = zip(*batch)
    return torch.stack(xs, dim=0), torch.stack(ys, dim=0)

# -------------------------
# ✅ ResNet Embedder
# -------------------------
class ResNetEmbedder(nn.Module):
    def __init__(self, emb_dim=256):
        super().__init__()
        base = resnet18(weights=None)
        base.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        base.maxpool = nn.Identity()
        self.backbone = nn.Sequential(*list(base.children())[:-1])  # [B, 512, 1, 1]
        self.fc = nn.Linear(512, emb_dim)

    def forward(self, x):
        h = self.backbone(x).flatten(1)  # [B, 512]
        e = self.fc(h)                   # [B, D]
        return F.normalize(e, dim=1)

# -------------------------
# ✅ DenseNet Embedder
# -------------------------
class DenseNetEmbedder(nn.Module):
    def __init__(self, emb_dim=256):
        super().__init__()
        base = densenet121(weights=None)
        base.features.conv0 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        base.features.pool0 = nn.Identity()

        self.features = base.features
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(base.classifier.in_features, emb_dim)

    def forward(self, x):
        f = self.features(x)
        f = F.relu(f, inplace=True)
        f = self.pool(f).flatten(1)      # [B, C]
        e = self.fc(f)                   # [B, D]
        return F.normalize(e, dim=1)

# -------------------------
# ✅ Weighted-sum Ensemble Embedder
# -------------------------
class WeightedEnsembleEmbedder(nn.Module):
    def __init__(self, r_embedder, d_embedder, alpha=0.8):
        super().__init__()
        self.r = r_embedder
        self.d = d_embedder
        self.alpha = alpha

    def forward(self, x):
        e_r = self.r(x)  # normalized
        e_d = self.d(x)  # normalized
        e = self.alpha * e_r + (1.0 - self.alpha) * e_d
        return F.normalize(e, dim=1)  # IMPORTANT

# -------------------------
# ✅ AM-Softmax Head
# -------------------------
class AMSoftmaxHead(nn.Module):
    def __init__(self, emb_dim, num_classes, s=30.0, m=0.2):
        super().__init__()
        self.W = nn.Parameter(torch.randn(num_classes, emb_dim))
        nn.init.xavier_normal_(self.W)
        self.s = s
        self.m = m

    def forward(self, emb, y):
        W = F.normalize(self.W, dim=1)   # [C, D]
        cos = F.linear(emb, W)           # [B, C]

        y_onehot = F.one_hot(y, num_classes=cos.size(1)).float()
        cos_m = cos - self.m * y_onehot
        logits = self.s * cos_m
        return logits

def split_indices(n, seed=42, train=0.8, val=0.1):
    idxs = list(range(n))
    rng = random.Random(seed)
    rng.shuffle(idxs)
    n_train = int(n * train)
    n_val = int(n * val)
    train_idxs = idxs[:n_train]
    val_idxs = idxs[n_train:n_train + n_val]
    test_idxs = idxs[n_train + n_val:]
    return train_idxs, val_idxs, test_idxs

def train_one_epoch(embedder, head, loader, optimizer, device, use_amp, epoch, model_name="Ensemble-Embed"):
    embedder.train()
    head.train()
    total_loss, steps = 0.0, 0
    scaler = torch.amp.GradScaler("cuda") if use_amp else None
    criterion = nn.CrossEntropyLoss()

    for i, batch in enumerate(loader):
        if batch is None:
            continue
        x, y = batch
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad(set_to_none=True)

        if use_amp:
            with torch.amp.autocast("cuda"):
                emb = embedder(x)
                logits = head(emb, y)
                loss = criterion(logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            emb = embedder(x)
            logits = head(emb, y)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

        total_loss += float(loss.item())
        steps += 1

        if (i + 1) % 50 == 0:
            print(f"[Epoch {epoch}][{model_name}] step {i+1}/{len(loader)} loss={total_loss/max(1,steps):.4f}")

    return total_loss / max(1, steps)

@torch.no_grad()
def extract_embeddings(embedder, loader, device):
    embedder.eval()
    embs, ys = [], []
    for batch in loader:
        if batch is None:
            continue
        x, y = batch
        x = x.to(device)
        e = embedder(x).cpu().numpy()
        embs.append(e)
        ys.append(y.numpy())
    if len(ys) == 0:
        return None
    return np.concatenate(embs, axis=0), np.concatenate(ys, axis=0)

def nearest_centroid_identification(embs_train, y_train, embs_eval, y_eval):
    num_classes = int(y_train.max()) + 1
    centroids = np.zeros((num_classes, embs_train.shape[1]), dtype=np.float32)

    for c in range(num_classes):
        m = embs_train[y_train == c].mean(axis=0)
        m = m / (np.linalg.norm(m) + 1e-12)
        centroids[c] = m

    sims = embs_eval @ centroids.T  # cosine sim since normalized
    preds = sims.argmax(axis=1)
    return {
        "acc": accuracy_score(y_eval, preds),
        "macro_f1": f1_score(y_eval, preds, average="macro"),
        "top1": float((preds == y_eval).mean())
    }

def compute_verification_eer(embs, y, max_pairs=30000):
    # 샘플링 기반 EER 근사
    n = len(y)
    rng = np.random.default_rng(42)

    scores = []
    labels = []
    for _ in range(max_pairs):
        i, j = rng.integers(0, n, size=2)
        if i == j:
            continue
        s = float(np.dot(embs[i], embs[j]))  # cosine (normalized)
        scores.append(s)
        labels.append(1 if y[i] == y[j] else 0)

    scores = np.array(scores)
    labels = np.array(labels)

    fpr, tpr, thr = roc_curve(labels, scores)
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.abs(fpr - fnr))
    eer = (fpr[eer_idx] + fnr[eer_idx]) / 2.0
    return {"eer": float(eer), "thr": float(thr[eer_idx])}

def main():
    os.makedirs("./data", exist_ok=True)

    raw = LIBRISPEECH("./data", url="dev-clean", download=True)

    all_speakers = sorted({int(spk) for _, _, _, spk, *_ in raw})
    speakers = all_speakers[:MAX_SPEAKERS]
    spk2idx = {spk: i for i, spk in enumerate(speakers)}
    num_classes = len(speakers)
    print(f"Using speakers: {num_classes}/{len(all_speakers)} | ALPHA={ALPHA}")

    full_ds = LibriSpeechSpeakerDataset(raw, spk2idx, n_mels=64, target_frames=256)

    tr, va, te = split_indices(len(full_ds), seed=42, train=0.8, val=0.1)
    train_ds = Subset(full_ds, tr)
    val_ds   = Subset(full_ds, va)
    test_ds  = Subset(full_ds, te)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    use_amp = (device.type == "cuda")

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, collate_fn=collate_skip_none)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_skip_none)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_skip_none)

    r_embed = ResNetEmbedder(emb_dim=EMB_DIM).to(device)
    d_embed = DenseNetEmbedder(emb_dim=EMB_DIM).to(device)
    embedder = WeightedEnsembleEmbedder(r_embed, d_embed, alpha=ALPHA).to(device)

    head = AMSoftmaxHead(emb_dim=EMB_DIM, num_classes=num_classes, s=SCALE, m=MARGIN).to(device)

    # ✅ 둘 다 학습시키는 경우(기본)
    optimizer = optim.Adam(list(r_embed.parameters()) + list(d_embed.parameters()) + list(head.parameters()), lr=1e-3)

    for ep in range(1, EPOCHS + 1):
        loss = train_one_epoch(embedder, head, train_loader, optimizer, device, use_amp, epoch=ep, model_name="WeightedEnsembleEmbed")
        print(f"\n[Epoch {ep}/{EPOCHS}] loss={loss:.4f}")

        pack_tr = extract_embeddings(embedder, train_loader, device)
        pack_va = extract_embeddings(embedder, val_loader, device)
        if pack_tr is not None and pack_va is not None:
            embs_tr, y_tr = pack_tr
            embs_va, y_va = pack_va
            val_id = nearest_centroid_identification(embs_tr, y_tr, embs_va, y_va)
            val_ver = compute_verification_eer(embs_va, y_va)
            print(f"  Val-ID  {val_id}")
            print(f"  Val-VER {val_ver}")

    pack_tr = extract_embeddings(embedder, train_loader, device)
    pack_te = extract_embeddings(embedder, test_loader, device)
    if pack_tr is None or pack_te is None:
        print("No embeddings extracted. Check data filtering.")
        return

    embs_tr, y_tr = pack_tr
    embs_te, y_te = pack_te
    test_id = nearest_centroid_identification(embs_tr, y_tr, embs_te, y_te)
    test_ver = compute_verification_eer(embs_te, y_te)

    print("\n=== TEST RESULTS (Embedding / Weighted Ensemble) ===")
    print(f"Test-ID  {test_id}")
    print(f"Test-VER {test_ver}")

if __name__ == "__main__":
    main()

100%|██████████| 322M/322M [00:18<00:00, 18.6MB/s]


Using speakers: 40/40 | ALPHA=0.8
